<a href="https://colab.research.google.com/github/AlfredoMijares/DeepLearning/blob/main/Santiago.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A3C (Asynchronous Advantage Actor-Critic) es un algoritmo de aprendizaje por refuerzo que se utiliza para enseñar a un agente a tomar decisiones en un entorno, buscando maximizar una recompensa acumulada. A3C es una extensión del método Actor-Critic, y su principal innovación es el uso de **entrenamiento asíncrono** para mejorar la estabilidad y eficiencia del aprendizaje.

A continuación, te explico los componentes clave de A3C sin usar ecuaciones:

### 1. **Actor-Critic**
El algoritmo A3C se basa en el enfoque **Actor-Critic**, que divide el proceso de aprendizaje en dos roles:

- **Actor**: El actor se encarga de **decidir qué acciones tomar** en base al estado del entorno. Es como un tomador de decisiones que elige una acción de un conjunto posible en cada momento.
  
- **Critic**: El crítico evalúa las decisiones del actor. Es como un evaluador que dice si las decisiones tomadas por el actor fueron buenas o malas en función del resultado. Lo hace estimando el valor de un estado o de una acción específica.

El actor y el crítico colaboran. El actor toma una acción, y el crítico evalúa cómo fue esa acción. Luego, el actor ajusta su comportamiento con base en la retroalimentación del crítico.

### 2. **Asíncrono (Entrenamiento en paralelo)**
Lo que hace único a A3C es que usa **múltiples agentes entrenando en paralelo** de forma **asíncrona** (independiente). En lugar de usar un solo agente para aprender de su experiencia, A3C utiliza **varios agentes que entrenan en diferentes copias del entorno al mismo tiempo**. Cada agente recoge experiencias y realiza actualizaciones, pero lo hace sin esperar a que otros agentes terminen, lo cual hace el proceso de aprendizaje mucho más rápido.

Esta técnica ayuda a reducir la **varianza** del aprendizaje y hace que el algoritmo sea más eficiente y estable.

### 3. **Ventaja**
En lugar de usar solo el valor del estado, A3C incorpora la idea de **ventaja** para mejorar el aprendizaje del actor. La ventaja es una medida que le dice al actor si una acción fue mejor o peor de lo esperado en comparación con otras posibles acciones en ese mismo estado.

- Si la ventaja es positiva, significa que la acción tomada fue **mejor** de lo esperado, por lo que el actor debería tomar acciones similares en el futuro.
- Si la ventaja es negativa, significa que la acción tomada fue **peor** de lo esperado, por lo que el actor debería cambiar su comportamiento.

### 4. **Actualización de los modelos**
El modelo de A3C tiene dos redes neuronales:
- Una red para el **actor** (que calcula las probabilidades de las acciones),
- Otra para el **crítico** (que estima los valores de los estados).

Cada vez que un agente toma una acción y recibe una recompensa, el modelo se actualiza, ajustando tanto la red del actor como la del crítico. Esto permite que el agente aprenda a elegir mejores acciones y a estimar con más precisión el valor de los estados.

### Resumen:
A3C mejora el algoritmo clásico Actor-Critic al usar múltiples agentes que entrenan en paralelo y de forma asíncrona. Cada agente interactúa con una copia del entorno, recolecta experiencias, y ajusta su comportamiento en base a la retroalimentación del crítico. Este enfoque mejora la eficiencia y estabilidad del aprendizaje en comparación con los métodos tradicionales.

En términos simples, A3C permite que varios "agentes" aprendan al mismo tiempo en diferentes entornos, se ayuden mutuamente y, a través de un proceso continuo de prueba y error, mejoren progresivamente.

In [1]:
pip install tensorflow gym numpy

In [3]:
import numpy as np
import tensorflow as tf
import gym
from tensorflow.keras import layers

# Definimos un modelo de red neuronal para el actor y el crítico
class A3CModel(tf.keras.Model):
    def __init__(self, action_space):
        super(A3CModel, self).__init__()

        # Red neuronal para el actor y el crítico
        self.dense1 = layers.Dense(128, activation='relu')
        self.dense2 = layers.Dense(128, activation='relu')

        # Capa para el valor de estado (Critic)
        self.value = layers.Dense(1, activation=None)

        # Capa para las probabilidades de las acciones (Actor)
        self.policy = layers.Dense(action_space, activation='softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)

        value = self.value(x)
        policy = self.policy(x)

        return policy, value

# Función para calcular la ventaja (Advantage)
def compute_advantage(reward, value, next_value, done, gamma=0.99, lambda_=0.95):
    delta = reward + gamma * next_value * (1 - done) - value
    advantage = delta  # Usamos delta como la ventaja en A3C simplificado
    return advantage

# Función de entrenamiento para el agente A3C
def train_a3c(env, model, optimizer, gamma=0.99):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        with tf.GradientTape() as tape:
            # Preprocesamiento de la entrada: convertimos el estado en un tensor
            state_tensor = tf.convert_to_tensor(state, dtype=tf.float32)
            state_tensor = tf.expand_dims(state_tensor, axis=0)  # Forma [1, tamaño_estado]

            # Pasamos el estado a través del modelo para obtener política y valor
            policy, value = model(state_tensor)

            # Tomamos una acción siguiendo la política (muestreo)
            action = np.random.choice(env.action_space.n, p=policy.numpy().flatten())

            # Realizamos la acción y obtenemos la nueva recompensa y estado
            next_state, reward, done, _ = env.step(action)
            total_reward += reward

            # Calculamos la ventaja
            next_state_tensor = tf.convert_to_tensor(next_state, dtype=tf.float32)
            next_state_tensor = tf.expand_dims(next_state_tensor, axis=0)
            _, next_value = model(next_state_tensor)

            advantage = compute_advantage(reward, value, next_value, done)

            # Calculamos las pérdidas para el actor y el crítico
            # Pérdida del crítico: error cuadrático medio
            critic_loss = tf.square(advantage)

            # Pérdida del actor: log-probabilidad de la acción * ventaja
            action_prob = policy[0, action]
            actor_loss = -tf.math.log(action_prob) * advantage

            # Pérdida total
            total_loss = actor_loss + critic_loss

        # Calculamos los gradientes y actualizamos los parámetros
        grads = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        state = next_state
    return total_reward

# Entrenamiento paralelo: en A3C original, usamos múltiples entornos y agentes, pero aquí
# solo mostramos un solo entorno para simplificar.
def main():
    # Inicializamos el entorno de OpenAI Gym (usaremos CartPole como ejemplo)
    env = gym.make("CartPole-v1")

    # Inicializamos el modelo A3C y el optimizador
    model = A3CModel(action_space=env.action_space.n)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Número de episodios de entrenamiento
    num_episodes = 1000

    # Bucle de entrenamiento
    for episode in range(num_episodes):
        total_reward = train_a3c(env, model, optimizer)
        if episode % 100 == 0:
            print(f'Episodio: {episode}, Recompensa total: {total_reward}')

    # Cerrar el entorno al final
    env.close()

if __name__ == '__main__':
    main()

Episodio: 0, Recompensa total: 49.0


KeyboardInterrupt: 

Descripción paso a paso del código:
Modelo A3C:

Clase A3CModel: Definimos una clase A3CModel que extiende tf.keras.Model. Este modelo tiene dos componentes:
Actor: Red neuronal que predice la probabilidad de cada acción (utilizando softmax en la salida).
Crítico: Red neuronal que predice el valor de estado (salida escalar).
Cálculo de la ventaja:

La función compute_advantage calcula la ventaja para una acción dada, lo que se utiliza para actualizar el modelo. Aquí se calcula una forma simplificada de la ventaja usando la diferencia entre la recompensa obtenida y el valor de estado predicho.
Entrenamiento:

La función train_a3c gestiona un único paso de entrenamiento, donde el agente selecciona una acción, observa el resultado y calcula la ventaja. Luego, se calcula la pérdida combinada de actor y crítico y se actualizan los parámetros de la red mediante retropropagación.
Bucle de entrenamiento:

En la función main, inicializamos el entorno de CartPole-v1 de OpenAI Gym y configuramos el modelo A3C y el optimizador (Adam).
En cada episodio, el agente entrena y actualiza su modelo. Imprimimos la recompensa total cada 100 episodios.
Explicación de los elementos clave:
Política (Actor): Utiliza una red neuronal para predecir las probabilidades de las acciones posibles en un estado dado.
Valor (Crítico): Utiliza una red neuronal para predecir la recompensa esperada en un estado dado.
Entrenamiento Asíncrono: Aunque este código no implementa múltiples agentes en paralelo (como en el A3C original), lo harías entrenando múltiples instancias del agente en diferentes entornos y luego combinando los gradientes.
Pérdida: La pérdida total es la suma de la pérdida del actor y la pérdida del crítico.
Este es un esquema básico y simplificado para mostrar cómo se puede construir un algoritmo A3C en Python con TensorFlow. El A3C completo incluye la paralelización de múltiples entornos y actores, lo que mejora significativamente la eficiencia, pero el código aquí mostrado está simplificado para fines de comprensión.

SAC (Soft Actor-Critic) es un algoritmo de **aprendizaje por refuerzo** basado en la técnica de **actor-crítico**, que busca mejorar la eficiencia y estabilidad del aprendizaje, especialmente en entornos con espacios de acción continuos. SAC se distingue por incorporar el concepto de **entropía** para hacer el aprendizaje más robusto y explorar más eficazmente el entorno.

### Conceptos clave de SAC:

1. **Actor-Critic**:
   - **Actor**: El actor elige qué acción tomar en cada momento. En SAC, la política del actor es **estocástica**, lo que significa que no siempre toma la misma acción en un estado dado, sino que tiene una distribución de probabilidades sobre las posibles acciones. Esto permite explorar el entorno de forma más amplia.
   - **Crítico**: El crítico evalúa la acción tomada por el actor, proporcionando una estimación de la "calidad" de esa acción (valor de la acción o valor de estado).

2. **Maximización de la Recompensa + Entropía**:
   - En SAC, el objetivo no es solo maximizar la recompensa, sino también **maximizar la entropía** de la política. La **entropía** mide el grado de **incertidumbre o aleatoriedad** en las decisiones del agente. Esto significa que SAC fomenta la exploración, evitando que el agente se quede atrapado en una política subóptima.
   - Al añadir la entropía como parte del objetivo, SAC no solo busca obtener recompensas más altas, sino también mantener una política más **exploratoria**, lo que ayuda a evitar que el agente se quede atascado en soluciones locales.

3. **Valor de la acción (Q-function)**:
   - SAC introduce dos **Q-functions** (funciones de valor de acción), lo que ayuda a estimar mejor el valor de las acciones en lugar de solo predecir el valor del estado. Esto permite una evaluación más precisa del beneficio de tomar una acción en un estado determinado.
   - Estas funciones de valor se estiman mediante redes neuronales, y se optimizan para reducir el error de predicción.

4. **Actualización de la política**:
   - La política en SAC se actualiza para maximizar tanto la **recompensa esperada** como la **entropía** de las decisiones. Esto se logra de manera eficiente mediante **optimización de gradientes**.
   - El **actor** es entrenado para elegir acciones que maximicen la recompensa esperada, mientras que se asegura de que la política mantenga una alta entropía, lo que lleva a una exploración efectiva.

5. **Temperatura**:
   - SAC utiliza un **parámetro de temperatura** que controla el peso que se le da a la entropía en la función objetivo. Este parámetro se ajusta automáticamente durante el entrenamiento para equilibrar la exploración y la explotación.

### Características principales de SAC:

- **Exploración efectiva**: Al incorporar la entropía en el objetivo de optimización, SAC fomenta la exploración de nuevas acciones, lo que evita que el agente se quede atrapado en políticas subóptimas.
  
- **Política estocástica**: A diferencia de otros algoritmos que utilizan políticas deterministas, SAC usa una política estocástica, lo que significa que, en un mismo estado, puede elegir diferentes acciones en distintas ocasiones. Esto ayuda a explorar mejor el espacio de soluciones.

- **Aprendizaje eficiente**: SAC es más **estable** y **robusto** en comparación con otros algoritmos de aprendizaje por refuerzo, como DDPG (Deep Deterministic Policy Gradient), especialmente cuando se trata de espacios de acción continuos.

- **Optimización de dos funciones de valor**: SAC utiliza **dos redes Q** para estimar el valor de las acciones, lo que ayuda a mejorar la estabilidad del entrenamiento.

- **Método off-policy**: SAC es un algoritmo **off-policy**, lo que significa que puede aprender de datos previamente recolectados, no solo de interacciones recientes con el entorno. Esto aumenta la eficiencia del proceso de aprendizaje.

### Resumen:

SAC es un algoritmo de aprendizaje por refuerzo que combina la **optimización de políticas estocásticas** con la **maximización de la entropía** para promover la exploración y mejorar la estabilidad del entrenamiento. Es especialmente útil en entornos con espacios de acción continuos y es conocido por ser más **robusto** y **eficiente** en comparación con otros métodos, como DDPG y A3C. Al incluir la entropía como parte del objetivo, SAC no solo busca maximizar las recompensas, sino también mantener una política que explore efectivamente el entorno.

In [7]:
pip install torch gym numpy

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym
from torch.distributions import Normal

# Red neuronal para el Actor y el Crítico
class SACModel(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(SACModel, self).__init__()

        # Capa compartida
        self.fc1 = nn.Linear(state_dim, 256)
        self.fc2 = nn.Linear(256, 256)

        # Red para la política (Actor)
        self.fc_actor_mean = nn.Linear(256, action_dim)  # Media de la acción
        self.fc_actor_logstd = nn.Linear(256, action_dim)  # Desviación estándar de la acción

        # Red para el Crítico (Q-value)
        self.fc_critic1 = nn.Linear(256 + action_dim, 256)
        self.fc_critic2 = nn.Linear(256, 1)

    def forward(self, state, action=None):
        x = torch.relu(self.fc1(state))  # Capa oculta
        x = torch.relu(self.fc2(x))  # Capa oculta

        # Actor: calcular media y log-desviación estándar
        mean = self.fc_actor_mean(x)
        logstd = self.fc_actor_logstd(x)
        std = torch.exp(logstd)

        # Si se proporciona una acción, calcular el valor de la acción (Q-value) para el Crítico
        if action is not None:
            x_with_action = torch.cat([x, action], dim=-1)  # Concatenamos el estado y la acción
            q_value = torch.relu(self.fc_critic1(x_with_action))
            q_value = self.fc_critic2(q_value)
            return mean, std, q_value

        return mean, std

# Función para elegir una acción según la política
def select_action(state, model):
    mean, std = model(state)  # Obtener la media y desviación estándar de la acción
    dist = Normal(mean, std)  # Distribución normal
    action = dist.sample()  # Muestrear una acción de la distribución
    return action, dist.log_prob(action).sum()  # Log-probabilidad de la acción elegida

# Entrenamiento del SAC
def train_sac(env, model, optimizer, num_episodes=1000, gamma=0.99, alpha=0.2):
    state = env.reset()  # Inicializamos el entorno
    state = torch.tensor(state, dtype=torch.float32)

    for episode in range(num_episodes):
        done = False
        total_reward = 0

        while not done:
            # Paso 1: Elegir una acción según la política (actor)
            action, log_prob = select_action(state, model)

            # Paso 2: Ejecutar la acción en el entorno
            next_state, reward, done, _ = env.step(action.numpy())  # Ejecutar acción
            next_state = torch.tensor(next_state, dtype=torch.float32)

            # Almacenar la transición (estado, acción, recompensa, siguiente estado)
            reward = torch.tensor(reward, dtype=torch.float32)

            # Paso 3: Actualizar el Crítico (Q-value)
            mean_next, std_next = model(next_state)  # Obtener la política para el siguiente estado
            next_action, _ = select_action(next_state, model)  # Acción para el siguiente estado
            next_q_value = model(next_state, next_action)[2]  # Calcular Q para el siguiente estado

            # Fórmula de Bellman para el valor objetivo de Q
            target_q_value = reward + gamma * next_q_value * (1 - done)
            q_value = model(state, action)[2]  # Valor Q para el estado actual

            # Error cuadrático medio para la función de valor (Crítico)
            critic_loss = torch.mean((q_value - target_q_value) ** 2)

            # Paso 4: Actualizar el Actor (Política)
            mean, std = model(state)  # Obtener la política actual
            dist = Normal(mean, std)
            log_prob = dist.log_prob(action).sum()  # Log-probabilidad de la acción

            # Maximizar la entropía y la recompensa (política suave)
            actor_loss = torch.mean(q_value - alpha * log_prob)

            # Paso 5: Optimizar el modelo
            optimizer.zero_grad()
            (critic_loss + actor_loss).backward()  # Gradientes combinados
            optimizer.step()  # Actualizamos los parámetros del modelo

            # Paso 6: Actualizar el estado
            state = next_state
            total_reward += reward.item()

        # Imprimir la recompensa total por episodio
        print(f"Episode {episode+1}, Total Reward: {total_reward}")

# Inicializar el entorno de OpenAI Gym
env = gym.make('Pendulum-v1')  # Usa la versión v1 del entorno Pendulum
state_dim = env.observation_space.shape[0]  # Dimensión del estado
action_dim = env.action_space.shape[0]  # Dimensión de la acción

# Inicializar el modelo y el optimizador
model = SACModel(state_dim, action_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenar el agente SAC
train_sac(env, model, optimizer)


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 1, Total Reward: -1424.3063924312592
Episode 2, Total Reward: -8.283285140991211
Episode 3, Total Reward: -8.317200660705566
Episode 4, Total Reward: -8.368592262268066
Episode 5, Total Reward: -8.435500144958496
Episode 6, Total Reward: -8.515868186950684
Episode 7, Total Reward: -8.606941223144531
Episode 8, Total Reward: -8.706811904907227
Episode 9, Total Reward: -8.813191413879395
Episode 10, Total Reward: -8.924426078796387
Episode 11, Total Reward: -9.037960052490234
Episode 12, Total Reward: -9.1524658203125
Episode 13, Total Reward: -9.26328182220459
Episode 14, Total Reward: -9.366769790649414
Episode 15, Total Reward: -9.470964431762695
Episode 16, Total Reward: -9.566863059997559
Episode 17, Total Reward: -9.670607566833496
Episode 18, Total Reward: -9.779193878173828
Episode 19, Total Reward: -9.8779296875
Episode 20, Total Reward: -9.796201705932617
Episode 21, Total Reward: -9.729433059692383
Episode 22, Total Reward: -9.720274925231934
Episode 23, Total Reward: 

Resumen
Este código implementa el algoritmo Soft Actor-Critic (SAC), que es un algoritmo de refuerzo profundo que optimiza tanto la política (actor) como la función de valor (crítico) de forma simultánea. El actor selecciona acciones de manera estocástica (para fomentar la exploración), mientras que el crítico evalúa las acciones y ayuda a mejorar la política.